# REI505M Machine Learning - Final project
### Due: --------

**Names**: Axel Kristján Axelsson, Bjarni Haukur Bjarnason <br />
**Email**: aka30@hi.is, bhb23@hi.is

**1. Image classification**

s


In [1]:
import numpy as np
from sklearn.model_selection import train_test_split
from PIL import Image
import tensorflow as tf

from data_loader import MyDataLoader
from neural_nets import NeuralNets
from helper_funcs import *


%load_ext autoreload 
%autoreload 2

norm = (128,128)

dl = MyDataLoader("combine2", norm)
nn = NeuralNets(norm)

In [9]:
dl.normalize_train_data(norm_size=norm)

try:
    del X
    del y

except:
    pass

X, y = dl.get_lab_data()
dl.numpy_dump(X, "X")
dl.numpy_dump(y, "y")

100%|██████████| 102/102 [01:29<00:00,  1.15it/s]


In [2]:
# X = np.squeeze(dl.numpy_load("X"))
X = dl.numpy_load("X") # 0-1
y = dl.numpy_load("y") # -1 - 1

In [3]:
X_lab = X
y_lab = map_to(y)

In [4]:
X_train, X_test, y_train, y_test = train_test_split(X_lab, y_lab, test_size = 0.3, random_state = 3)
X_test, X_val, y_test, y_val = train_test_split(X_test, y_test, test_size = 0.33, random_state = 3)

In [11]:
# from random import randint
from tensorflow.keras.optimizers import *
from tensorflow.keras.losses import *

# ?GANS svona
# gpu_options = tf.compat.v1.GPUOptions(per_process_gpu_memory_fraction=0.6)
# sess = tf.compat.v1.Session(config=tf.compat.v1.ConfigProto(gpu_options=gpu_options))

log_dir = "logs/fit/" + "model_e"
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

model = load_model("model_e")
#model = nn.model_e()
model.compile(optimizer=RMSprop(1e-4), loss=MeanSquaredError())



model.fit(X_train, y_train, batch_size = 64, epochs = 30, validation_data=(X_val, y_val), callbacks=[tensorboard_callback])#, callbacks=[callback])
#save_model(model, "model_e", brave=True)



Epoch 1/30
128/128 [==============================] - 7s 48ms/step - loss: 0.0051 - val_loss: 0.0064
Epoch 2/30
128/128 [==============================] - 5s 41ms/step - loss: 0.0051 - val_loss: 0.0064
Epoch 3/30
128/128 [==============================] - 5s 41ms/step - loss: 0.0050 - val_loss: 0.0064
Epoch 4/30
128/128 [==============================] - 5s 41ms/step - loss: 0.0050 - val_loss: 0.0064
Epoch 5/30
128/128 [==============================] - 5s 41ms/step - loss: 0.0050 - val_loss: 0.0065
Epoch 6/30
128/128 [==============================] - 5s 39ms/step - loss: 0.0050 - val_loss: 0.0065
Epoch 7/30
128/128 [==============================] - 5s 39ms/step - loss: 0.0050 - val_loss: 0.0064
Epoch 8/30
128/128 [==============================] - 5s 38ms/step - loss: 0.0050 - val_loss: 0.0065
Epoch 9/30
128/128 [==============================] - 5s 40ms/step - loss: 0.0050 - val_loss: 0.0065
Epoch 10/30
128/128 [==============================] - 5s 36ms/step - loss: 0.0049 - val_lo

In [12]:
%tensorboard --logdir logs/fit/

UsageError: Line magic function `%tensorboard` not found.


In [13]:
rand_array = np.random.choice(X_test.shape[0], 11)
pred = model.predict(X_test[rand_array])
show_images(X_test[rand_array], map_from(pred))

C:\Users\bjarn\AppData\Roaming\Python\Python39\site-packages\skimage\color\colorconv.py:1109: UserWarning: Color data out of range: Z < 0 in 6 pixels
  return xyz2rgb(lab2xyz(lab, illuminant, observer))
C:\Users\bjarn\AppData\Roaming\Python\Python39\site-packages\skimage\color\colorconv.py:1109: UserWarning: Color data out of range: Z < 0 in 9 pixels
  return xyz2rgb(lab2xyz(lab, illuminant, observer))


In [ ]:
print(X_lab.shape)
print(y_lab.shape)
print(np.max(X_lab))
print(np.max(y_lab))
print(np.min(X_lab))
print(np.min(y_lab))
